##### Importing Required Libraries

In [1]:
import pandas as pd
import numpy as np

##### Fetch the data from url to dataframe

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

dataset=pd.read_html(url, header=0)[0]

dataset.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [3]:
dataset.shape

(288, 3)

In [4]:
dataset['Postcode'][0].index
dataset["Latitude"] = np.nan
dataset["Longitude"] = np.nan

In [5]:
geodata = pd.read_csv('https://cocl.us/Geospatial_data')
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
i = 0
for postalcode in dataset['Postcode']:
    j = 0
    for pcode in geodata['Postal Code']:
        if postalcode == pcode :
            dataset['Latitude'][i] = geodata['Latitude'][j]
            dataset['Longitude'][i] = geodata['Longitude'][j]
        j = j + 1
    i = i + 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [7]:
dataset = dataset.dropna()
dataset = dataset.reset_index(drop=True)
dataset.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [8]:
toronto_data = dataset[dataset['Borough'].str.contains("Toronto")]
toronto_data.drop(columns=['Postcode'], inplace=True)
toronto_data.reset_index(inplace=True, drop=True)
toronto_data.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Borough,Neighbourhood,Latitude,Longitude
0,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,Downtown Toronto,Regent Park,43.654260,-79.360636
2,Downtown Toronto,Ryerson,43.657162,-79.378937
3,Downtown Toronto,Garden District,43.657162,-79.378937
4,Downtown Toronto,St. James Town,43.651494,-79.375418


In [9]:
from geopy.geocoders import Nominatim
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [10]:
import folium
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Explore  Neighbourhood

In [11]:
CLIENT_ID = 'L5GQQYIOZ2FY3KII3YW1QRKLFOLCR3SVJQ5P5B1HI4UMDISI' # your Foursquare ID
CLIENT_SECRET = 'CNVTYOSDSD1M4H5FHJ2YNOIPUWDHAAKA3UWZ2BQ2BWXMIAL3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: L5GQQYIOZ2FY3KII3YW1QRKLFOLCR3SVJQ5P5B1HI4UMDISI
CLIENT_SECRET:CNVTYOSDSD1M4H5FHJ2YNOIPUWDHAAKA3UWZ2BQ2BWXMIAL3


In [12]:
import requests
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 10
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city


In [14]:
print(toronto_venues.shape)
toronto_venues.head()

(687, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [15]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adelaide,10,10,10,10,10,10
Bathurst Quay,10,10,10,10,10,10
Berczy Park,10,10,10,10,10,10
Brockton,10,10,10,10,10,10
Business Reply Mail Processing Centre 969 Eastern,10,10,10,10,10,10
CN Tower,10,10,10,10,10,10
Cabbagetown,10,10,10,10,10,10
Central Bay Street,10,10,10,10,10,10
Chinatown,10,10,10,10,10,10


In [16]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 116 uniques categories.


In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Workshop,...,Swim School,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
toronto_onehot.shape

(687, 117)

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Workshop,...,Swim School,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Adelaide,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.1,0.0,0.0,0.0
1,Bathurst Quay,0.1,0.1,0.1,0.2,0.1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.1,0.0,0.0,0.00,0.1,0.0,0.0,0.0
3,Brockton,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
4,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
5,CN Tower,0.1,0.1,0.1,0.2,0.1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
6,Cabbagetown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
7,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
8,Chinatown,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.1,0.0,0.0
9,Christie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0


In [22]:
toronto_grouped.shape

(73, 117)

In [33]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                venue  freq
0               Plaza   0.1
1        Concert Hall   0.1
2  Seafood Restaurant   0.1
3          Steakhouse   0.1
4         Opera House   0.1


----Bathurst Quay----
                venue  freq
0      Airport Lounge   0.2
1             Airport   0.1
2            Boutique   0.1
3  Airport Food Court   0.1
4               Plane   0.1


----Berczy Park----
             venue  freq
0     Concert Hall   0.1
1  Thai Restaurant   0.1
2             Park   0.1
3   Farmers Market   0.1
4           Museum   0.1


----Brockton----
            venue  freq
0     Coffee Shop   0.2
1             Gym   0.1
2             Bar   0.1
3            Café   0.1
4  Breakfast Spot   0.1


----Business Reply Mail Processing Centre 969 Eastern----
            venue  freq
0         Brewery   0.1
1  Farmers Market   0.1
2     Pizza Place   0.1
3      Skate Park   0.1
4   Garden Center   0.1


----CN Tower----
                venue  freq
0      Airport Lounge   0.2
1        

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Opera House,Asian Restaurant,Speakeasy,Steakhouse,Hotel,Plaza,Seafood Restaurant,Concert Hall,Greek Restaurant,Vegetarian / Vegan Restaurant
1,Bathurst Quay,Airport Lounge,Airport,Boutique,Harbor / Marina,Plane,Coffee Shop,Airport Food Court,Airport Gate,Airport Terminal,American Restaurant
2,Berczy Park,Beer Bar,Concert Hall,Vegetarian / Vegan Restaurant,Park,Museum,Thai Restaurant,French Restaurant,Farmers Market,Liquor Store,Steakhouse
3,Brockton,Coffee Shop,Breakfast Spot,Furniture / Home Store,Caribbean Restaurant,Café,Bar,Italian Restaurant,Pet Store,Gym,French Restaurant
4,Business Reply Mail Processing Centre 969 Eastern,Pizza Place,Auto Workshop,Restaurant,Burrito Place,Farmers Market,Skate Park,Brewery,Fast Food Restaurant,Comic Shop,Garden Center


# Clustering

In [43]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 3, 2, 1, 2, 3, 1, 1, 1, 1])

In [44]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Harbourfront,43.654260,-79.360636,1,Breakfast Spot,Spa,Historic Site,Coffee Shop,Pub,Restaurant,Bakery,Park,Gym / Fitness Center,French Restaurant
1,Downtown Toronto,Regent Park,43.654260,-79.360636,1,Breakfast Spot,Spa,Historic Site,Coffee Shop,Pub,Restaurant,Bakery,Park,Gym / Fitness Center,French Restaurant
2,Downtown Toronto,Ryerson,43.657162,-79.378937,2,Comic Shop,Thai Restaurant,Burrito Place,Ramen Restaurant,Plaza,Café,Tea Room,Pizza Place,Theater,Clothing Store
3,Downtown Toronto,Garden District,43.657162,-79.378937,2,Comic Shop,Thai Restaurant,Burrito Place,Ramen Restaurant,Plaza,Café,Tea Room,Pizza Place,Theater,Clothing Store
4,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Café,Restaurant,Japanese Restaurant,Italian Restaurant,Coffee Shop,Food Truck,Creperie,Indian Restaurant,Bakery,Jewelry Store


In [45]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,Moore Park,Restaurant,Gym,Trail,Summer Camp,Yoga Studio,Farmers Market,Eastern European Restaurant,Dog Run,Diner,Dance Studio
50,Summerhill East,Restaurant,Gym,Trail,Summer Camp,Yoga Studio,Farmers Market,Eastern European Restaurant,Dog Run,Diner,Dance Studio


In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Harbourfront,Breakfast Spot,Spa,Historic Site,Coffee Shop,Pub,Restaurant,Bakery,Park,Gym / Fitness Center,French Restaurant
1,Regent Park,Breakfast Spot,Spa,Historic Site,Coffee Shop,Pub,Restaurant,Bakery,Park,Gym / Fitness Center,French Restaurant
4,St. James Town,Café,Restaurant,Japanese Restaurant,Italian Restaurant,Coffee Shop,Food Truck,Creperie,Indian Restaurant,Bakery,Jewelry Store
7,Central Bay Street,Coffee Shop,Sushi Restaurant,Italian Restaurant,Modern European Restaurant,Ramen Restaurant,Spa,Pizza Place,Gastropub,Cosmetics Shop,Creperie
8,Christie,Café,Grocery Store,Italian Restaurant,Diner,Candy Store,Restaurant,Coffee Shop,Arts & Crafts Store,Asian Restaurant,Cuban Restaurant
12,Dovercourt Village,Bakery,Pharmacy,Brewery,Bar,Supermarket,Café,Middle Eastern Restaurant,Music Venue,Gym / Fitness Center,Dessert Shop
13,Dufferin,Bakery,Pharmacy,Brewery,Bar,Supermarket,Café,Middle Eastern Restaurant,Music Venue,Gym / Fitness Center,Dessert Shop
21,Design Exchange,Coffee Shop,Pub,Gastropub,Gym,Gym / Fitness Center,Café,Restaurant,Hotel,Beer Bar,Dog Run
22,Toronto Dominion Centre,Coffee Shop,Pub,Gastropub,Gym,Gym / Fitness Center,Café,Restaurant,Hotel,Beer Bar,Dog Run
23,Brockton,Coffee Shop,Breakfast Spot,Furniture / Home Store,Caribbean Restaurant,Café,Bar,Italian Restaurant,Pet Store,Gym,French Restaurant


In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Ryerson,Comic Shop,Thai Restaurant,Burrito Place,Ramen Restaurant,Plaza,Café,Tea Room,Pizza Place,Theater,Clothing Store
3,Garden District,Comic Shop,Thai Restaurant,Burrito Place,Ramen Restaurant,Plaza,Café,Tea Room,Pizza Place,Theater,Clothing Store
5,The Beaches,Trail,Neighborhood,Health Food Store,Pub,Yoga Studio,Fast Food Restaurant,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant
6,Berczy Park,Beer Bar,Concert Hall,Vegetarian / Vegan Restaurant,Park,Museum,Thai Restaurant,French Restaurant,Farmers Market,Liquor Store,Steakhouse
9,Adelaide,Opera House,Asian Restaurant,Speakeasy,Steakhouse,Hotel,Plaza,Seafood Restaurant,Concert Hall,Greek Restaurant,Vegetarian / Vegan Restaurant
10,King,Opera House,Asian Restaurant,Speakeasy,Steakhouse,Hotel,Plaza,Seafood Restaurant,Concert Hall,Greek Restaurant,Vegetarian / Vegan Restaurant
11,Richmond,Opera House,Asian Restaurant,Speakeasy,Steakhouse,Hotel,Plaza,Seafood Restaurant,Concert Hall,Greek Restaurant,Vegetarian / Vegan Restaurant
14,Harbourfront East,Park,Performing Arts Venue,Dessert Shop,Neighborhood,Sporting Goods Shop,Salad Place,Plaza,Lake,Hotel,Supermarket
15,Toronto Islands,Park,Performing Arts Venue,Dessert Shop,Neighborhood,Sporting Goods Shop,Salad Place,Plaza,Lake,Hotel,Supermarket
16,Union Station,Park,Performing Arts Venue,Dessert Shop,Neighborhood,Sporting Goods Shop,Salad Place,Plaza,Lake,Hotel,Supermarket


In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,CN Tower,Airport Lounge,Airport,Boutique,Harbor / Marina,Plane,Coffee Shop,Airport Food Court,Airport Gate,Airport Terminal,American Restaurant
60,Bathurst Quay,Airport Lounge,Airport,Boutique,Harbor / Marina,Plane,Coffee Shop,Airport Food Court,Airport Gate,Airport Terminal,American Restaurant
61,Island airport,Airport Lounge,Airport,Boutique,Harbor / Marina,Plane,Coffee Shop,Airport Food Court,Airport Gate,Airport Terminal,American Restaurant
62,Harbourfront West,Airport Lounge,Airport,Boutique,Harbor / Marina,Plane,Coffee Shop,Airport Food Court,Airport Gate,Airport Terminal,American Restaurant
63,King and Spadina,Airport Lounge,Airport,Boutique,Harbor / Marina,Plane,Coffee Shop,Airport Food Court,Airport Gate,Airport Terminal,American Restaurant
64,Railway Lands,Airport Lounge,Airport,Boutique,Harbor / Marina,Plane,Coffee Shop,Airport Food Court,Airport Gate,Airport Terminal,American Restaurant
65,South Niagara,Airport Lounge,Airport,Boutique,Harbor / Marina,Plane,Coffee Shop,Airport Food Court,Airport Gate,Airport Terminal,American Restaurant


In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Roselawn,Garden,Yoga Studio,Fish Market,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Dessert Shop,Diner
